## First we initialize our globals

Inititalizes the `db.cow_bff` table with sample cow eating time data.

Run this notebook from the web UI or by executing the `Databricks: Run File as Workflow on Databricks` command.

In [ ]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [8]:
import random

start_epoch = 1684540800
end_epoch = 1685620800
day = 86400

cows = ['Butterscotch','Hershey','Mocha','Nutella','Brandy','Peaches','Marshmallow','Popcorn','Muffin','Daisy','Buttercup','Dottie','Magic','Nellie',]

meal_1_start = 32400

#we will only focus on one meal
meal_2_start = 64800

meal_duration_start = -600
meal_duration_end = 2400



In [9]:
index = start_epoch
import pandas as pd
import datetime
import calendar
meal_data2 = None
data =[]
while True:
  if index > end_epoch:
    break;
  date = datetime.datetime.fromtimestamp(index)
  for cow in cows:
    r1 = random.randint(meal_duration_start, meal_duration_end)
    r2 = random.randint(meal_duration_start, meal_duration_end)
    begin = r1
    end = r2
    if r1 > r2:
      begin = r2
      end = r1
    meal_start_seconds = round(meal_1_start+begin)
    meal_end_seconds = round(meal_1_start+end)
    meal_data1 = {
      'cow_name': cow,
      'meal_start': meal_start_seconds, 
      'meal_end': meal_end_seconds,
      'meal_start_time':str(datetime.timedelta(seconds=meal_start_seconds)), 
      'meal_end_time':str(datetime.timedelta(seconds=meal_end_seconds)),
      'duration': meal_end_seconds-meal_start_seconds,
      'date': date,
      'day': calendar.day_name[date.weekday()]}
    data.append(meal_data1)
    # we cheat and add Mocha's best friend Cocoa
    if(cow=='Mocha'):
      cocoa_meal_start_seconds = round(meal_start_seconds - (random.randint(0,500)/10000.0))
      cocoa_meal_end_seconds = round(meal_end_seconds- (random.randint(0,500)/10000.0))
      meal_data2 = {
        'cow_name': 'Cocoa',
        'meal_start': cocoa_meal_start_seconds, 
        'meal_end': cocoa_meal_end_seconds,
        'meal_start_time': str(datetime.timedelta(seconds=cocoa_meal_start_seconds)), 
        'meal_end_time': str(datetime.timedelta(seconds=cocoa_meal_end_seconds)),
        'duration': cocoa_meal_end_seconds-cocoa_meal_start_seconds,
        'date': date,
        'day': calendar.day_name[date.weekday()]}
      data.append(meal_data2)
    
  index = index + day
  df = pd.DataFrame(data)
  

In [1]:
%sql
use catalog field_demos;

""


In [2]:
%sql
create schema if not exists db;

""


In [5]:
%sql
drop table if exists db.cows_bff;

""


In [10]:
# # save data into table
spark_df = spark.createDataFrame(df)

# Write Frame out as Table
spark_df.write.mode("overwrite").saveAsTable("db.cows_bff")

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/connect/session.py:368: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64_dtype(t) or is_datetime64tz_dtype(t)
/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/serializers.py:256: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(series.dtype):


In [12]:
cows_bff = spark.table("field_demos.db.cows_bff")
display(cows_bff)

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:567: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not is_datetime64tz_dtype(pser.dtype):
/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:386: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):


,cow_name,meal_start,meal_end,meal_start_time,meal_end_time,duration,date,day
0,Butterscotch,31847,33385,8:50:47,9:16:25,1538,2023-05-19 17:00:00,Friday
1,Hershey,32162,34594,8:56:02,9:36:34,2432,2023-05-19 17:00:00,Friday
2,Mocha,33165,33453,9:12:45,9:17:33,288,2023-05-19 17:00:00,Friday
3,Cocoa,33165,33453,9:12:45,9:17:33,288,2023-05-19 17:00:00,Friday
4,Nutella,31805,34695,8:50:05,9:38:15,2890,2023-05-19 17:00:00,Friday
5,Brandy,31960,33923,8:52:40,9:25:23,1963,2023-05-19 17:00:00,Friday
6,Peaches,32778,32942,9:06:18,9:09:02,164,2023-05-19 17:00:00,Friday
7,Marshmallow,32095,32995,8:54:55,9:09:55,900,2023-05-19 17:00:00,Friday
8,Popcorn,32108,32416,8:55:08,9:00:16,308,2023-05-19 17:00:00,Friday
9,Muffin,31970,32924,8:52:50,9:08:44,954,2023-05-19 17:00:00,Friday


In [13]:
import plotly.express as px

pdf = cows_bff.toPandas().groupby("cow_name", as_index=False).agg({"duration": "mean"})

fig = px.bar(pdf, x='cow_name', y='duration')
fig.show()

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:567: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not is_datetime64tz_dtype(pser.dtype):
/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:386: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
